# 웹툰 정보 가져오기

In [ ]:
import csv

f = open('webtoon_info.csv','w')
wr = csv.writer(f)

In [ ]:
import requests
from bs4 import BeautifulSoup
from itertools import chain
from collections import defaultdict
import csv

def get_week(soup) :
    # 요일별 전체 웹툰 가져오기
    title = soup.select('.title')
    t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title))

    new_week = []
    for v in t_weekdays:
        if v not in new_week:
            new_week.append(v)
            
#     print(new_week)
    return(new_week)

def main():
    list_href = []

    url = "https://comic.naver.com/webtoon/weekday.nhn"
    result = requests.get(url)
    soup = BeautifulSoup(result.text, "html.parser")

    # 요일 가져오기
    week = get_week(soup)
    
    #웹툰별 스코어
    score_info = get_score(week)

    
    #웹툰 코드와 제목
    code_title = get_code(soup)
    
    # 코드만 분류
    code = list(code_title.keys())

    
    # {코드: [제목, 점수]}
    code_title_score = {}
    for c in code:
        code_title_score[c] = [score_info.get(c) , code_title.get(c)]
    
    # {코드 :{점수, 제목, 추가 정보 ~~~~~}}
    total_info = {}    
    for c in code:
        total_info[c] = [code_title_score.get(c) , get_info(c)]

    detail_list = list(total_info.values())
    
    for n in range(len(code)):
        wr.writerow([code[n], detail_list[n]])
    
if __name__ == "__main__":
    main()# 요일별 전체 웹툰 가져오기

In [ ]:
f.close()

# 코드따오기

In [ ]:
def get_code(soup):
    title = soup.select('.title')
    
    # 웹툰 코드, 요일별, 웹툰 제목
    t_Code=list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title))
    t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title))
    t_names = list(map(lambda x: x.text ,title))
    
    
    # data = {웹툰 코드 : 웹툰 제목}
    data ={}
    for i in range(len(t_Code)):
        data[t_Code[i]] =t_names[i]
        
        
    
    new_week = []
    for v in t_weekdays:
        if v not in new_week:
            new_week.append(v)
#     print(new_week)
    
    score = get_score(new_week)

    return(data)

# 평점가져오기

In [ ]:
def get_score(week_list):
    score_data = {}
    for week in week_list:
        score_url = "https://comic.naver.com/webtoon/weekdayList.nhn?week=" + week
        score_req = requests.get(score_url)
        score_soup = BeautifulSoup(score_req.text, "html.parser")

        title_info = score_soup.select('ul.img_list > li > dl > dt > a')
        t_code=list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title_info))
#         print(t_IDs)

        score_info = score_soup.select('div.rating_type > strong')
        t_score = list(map(lambda x: x.text ,score_info))
#         print(t_score)
    
        for i in range(len(t_code)):
            score_data[t_code[i]]= t_score[i]
    

#     print(score_data)
    return(score_data)


# 작가 이름들, 소개글, 첫회가기, 장르

In [ ]:
def get_info(code):
    wt_url = ("https://comic.naver.com/webtoon/list.nhn?titleId="+str(code))
    req = requests.get(wt_url)
    soup = BeautifulSoup(req.text, "html.parser")
    # print(soup)

    web_info= soup.select('div.comicinfo')
    # print(web_info)
    
    
    genre = ''
    writer = ''
    detail = ''
    
    webtoon_info = []
    for info in web_info:
        
        detail_tag = info.select_one('div.detail')
        genre = detail_tag.select_one('p.detail_info > span.genre').text

        name = info.select_one('h2 > span.wrt_nm').text
        
        writer = name.strip()

        detail = detail_tag.find('p').text

    # 첫 회 보기

    first_url = 'https://comic.naver.com/webtoon/detail.nhn?titleId='+str(code)+'&no=1'
    
    
    return([writer, genre, detail, first_url])